# Data management

## Unsupervised Learning

## [Michel Coppée](https://www.uliege.be/cms/c_9054334/fr/repertoire?uid=u224042) & [Malka Guillot](https://malkaguillot.github.io/)

## HEC Liège | [ECON2306]()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import seaborn as sns; 
sns.set_theme()

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

from scipy.cluster import hierarchy

%matplotlib inline
plt.style.use('seaborn-white')


## Principal Component Analysis

In scikit-learn, PCA is implemented as a `transformer` object that learns $n$ components in its `fit` method, and can be used on new data to project it on these components

In [ ]:
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt

### Generate Data

In [ ]:
#generate some data
X = np.random.normal(0, 1, (100, 4))
X[:,2] = 3 * X[:,0] - 2 * X[:,1] + np.random.normal(0, 0.1, 100)
X[:,3] = 1.5 * X[:,0] - 0.5 * X[:,1] + np.random.normal(0, 0.1, 100)

In [ ]:
X.shape

In [ ]:
#each feature will have zero mean
X = X - np.mean(X, axis=0)

In [ ]:
plt.figure(figsize=(10,10))
for i in range(4):
    for j in range(4):
        if j > i:
            plt.subplot(4,4,i*4+j+1)
            plt.scatter(X[:,i], X[:,j])
            plt.xlabel(f'x{i+1}', fontsize=20)
            plt.ylabel(f'x{j+1}', fontsize=20)
plt.tight_layout()

#### Observations:
- x1 and x2 do not seem correlated
- x1 seems very correlated with both x3 and x4
- x2 seems somewhat correlated with both x3 and x4
- x3 and x4 seem very correlated

### PCA minimal working example

Note: `PCA` centers the input data automatically. To scale the variables so that they have a unit variance, you can use the optional parameter `whiten=True`. 

#### Get PCA Components

We use the `PCA` module from `sllearn` to form the Principal Components

Then, we can project the data into the new feature space.


In [ ]:
#initialize; reduce to 3 features with PCA (from 4)
pca = PCA(n_components=3, whiten=True)

In [ ]:
# # Fit and transform data
pca_features = pca.fit_transform(X)
pca_features[:5]

In [ ]:
len(pca_features) # for each observation, we have the coordinates in 3D

In [ ]:
# Create dataframe
pca_df = pd.DataFrame(
    data=pca_features, 
    columns=['PC1', 'PC2', 'PC3'])
pca_df.head()

In [ ]:
fig = plt.figure(figsize = (10, 7))

ax = plt.axes(projection="3d")
scatter_plot= ax.scatter(X[:,1], X[:,0], X[:,2], c=X[:,3], cmap='Greens');
plt.colorbar(scatter_plot)
ax.set_xlabel('X1')
ax.set_ylabel('X0')
ax.set_zlabel('X2')
ax.zaxis.labelpad=-0.8
plt.show()

### PCA model attribute plots

PCA components and their significance can be explained using following attributes

- **Explained variance** is the amount of variance explained by each of the selected components. This attribute is associated with the sklearn PCA model as `explained_variance_`

- **Explained variance ratio** is the percentage of variance explained by each of the selected components. It’s attribute is `explained_variance_ratio_`

In [ ]:
pca.explained_variance_ 

In [ ]:
pca.explained_variance_ratio_

#### `explained_variance_` plot

In [ ]:
plt.bar(range(1,len(pca.explained_variance_ )+1),pca.explained_variance_ )
plt.ylabel('Explained variance')
plt.xlabel('Components')
plt.plot(range(1,len(pca.explained_variance_ )+1),
         np.cumsum(pca.explained_variance_),
         c='red',
         label="Cumulative Explained Variance")
plt.legend(loc='upper left')

#### `explained_variance_ratio_` plot

In [ ]:
plt.figure(figsize=(7,5))
plt.plot([1,2,3], pca.explained_variance_ratio_, '-o', label='Individual component')
plt.plot([1,2,3], np.cumsum(pca.explained_variance_ratio_), '-s', label='Cumulative')
plt.ylabel('Proportion of Variance Explained')
plt.xlabel('Principal Component')
plt.xlim(0.75,4.25)
plt.ylim(0,1.05)
plt.xticks([1,2,3])
plt.legend();
plt.show()

#### 2D Scatter plot of PC1 and PC2

In [ ]:
plt.scatter(pca_df['PC1'], pca_df['PC2'])

#### 3D Scatter plot of PC1,PC2 and PC3

In [ ]:
fig = plt.figure(figsize = (10, 7))
ax = plt.axes(projection="3d")
scatter_plot= ax.scatter(pca_df['PC1'], pca_df['PC2'],pca_df['PC3']);
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')
ax.zaxis.labelpad=-0.8
plt.show()

In [ ]:
#Make Plotly figure
import plotly.express as px
import plotly.graph_objects as go

fig = px.scatter_3d(x=pca_df['PC1'],
                    y=pca_df['PC2'],
                    z=pca_df['PC3'], color=pca_df['PC3'], opacity=0.7,
                    )
fig.update_layout(
    scene = dict(
                    xaxis_title='PC1',
                    yaxis_title='PC2',
                    zaxis_title='PC3'),
                    width=700,
margin=dict(l=0, r=0, b=0, t=0))
fig.show()

In [ ]:
# Principal components correlation coefficients
loadings = pca.components_
loadings

In [ ]:
# Number of features before PCA
n_features = pca.n_features_in_
 
# Feature names before PCA
feature_names = [0, 1, 2, 3]
 
# PC names
pc_list = [f'PC{i}' for i in list(range(1, n_features + 1))]
 
# Match PC names to loadings
pc_loadings = dict(zip(pc_list, loadings))
 
# Matrix of corr coefs between feature names and PCs
loadings_df = pd.DataFrame.from_dict(pc_loadings)
loadings_df['feature_names'] = feature_names
loadings_df = loadings_df.set_index('feature_names')
loadings_df

#### Plotting the correlation coefficients (loadings) of each feature.

In [ ]:
# Get the loadings of x and y axes
xs = loadings[0]
ys = loadings[1]
zs = loadings[2]
 
fig = plt.figure(figsize = (10, 7))
ax = plt.axes(projection="3d")

scatter_plot= ax.scatter(pca_df['PC1'], pca_df['PC2'],pca_df['PC3'], 
    c=pca_df['PC3'],
    cmap='Greens', 
    alpha=0.5)

fig = px.scatter_3d(x=pca_df['PC1'],
                    y=pca_df['PC2'],
                    z=pca_df['PC3'], color=pca_df['PC3'], opacity=0.7,
                    )

# Plot the loadings on a scatterplot
for i, varnames in enumerate(feature_names):
    ax.scatter(xs[i], ys[i], zs[i], s=100)
    ax.text(xs[i], ys[i],zs[i], varnames)
    ax.quiver(0, 0, 0, xs[i], ys[i],zs[i], color='red',arrow_length_ratio=0.1)  

ax.set_xlabel('PC1', rotation=-10)
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')

# Show plot
plt.title('3D Loading plot')
plt.show()

## `USAarrests` data


For each of the 50 states in the United States, the data set contains 
- the number of arrests per $100, 000$ residents 
- for each of three crimes: `Assault` ,` Murder` ,and `Rape` . 
- Other variable: `UrbanPop` (the % of the population in each state living in urban areas).

In [ ]:
import os
parent_path=os.path.dirname(os.getcwd()) # os.getcwd() fetchs the current path, 
data_path=os.path.join(parent_path, 'data')
df = pd.read_csv(os.path.join(data_path, 'USArrests.csv'), index_col=0)
df.shape

In [ ]:
df.head()

### Pre-processing
PCA should be performed after standardizing each variable to have mean zero and standard deviation one.

In [ ]:
from sklearn.preprocessing import scale

X = pd.DataFrame(scale(df), index=df.index, columns=df.columns)
X.head()

In [ ]:
X.shape

## 3D scattered points

<div class="alert alert-info">
<h3> Your turn: Plot the 3D scatterplot of the data, with color intensity for the fourth variable </h3>
</div>

<div class="alert alert-info">
<h3> Your turn:  </h3>

- Initialize a PCA with 4 components 
- `Fit` the PCA model and `transform` X to get the principal components 
- Compute the loading vectors 

</div>

<div class="alert alert-info">
<h3> Your turn: Plot the 3D scatterplot of the pca features </h3>
</div>

<div class="alert alert-info">
<h3> Your turn: plot the proportion of Variance Explained by principal component </h3>

How many components do you chose? 
</div>

## Clustering Methods
<html><div style='float:left'></div><hr color='#EB811B' size=1px width=796px></html>

### Generate a two-dimensional dataset containing four distinct blobs

In [ ]:
#from sklearn.datasets.samples_generator import make_blobs # random datapoints in 2D
from sklearn.datasets import make_blobs

X, y_true = make_blobs(n_samples=300, centers=4,
                       cluster_std=0.60, random_state=0)
plt.figure(figsize=(8, 5))
plt.scatter(X[:, 0], X[:, 1], s=50);
plt.show()

### Means Clustering

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=4, random_state=42)
kmeans.fit(X)
y_kmeans = kmeans.predict(X)
#y_kmeans # Each instance was assigned to one of the 4 clusters:

In [ ]:
kmeans.cluster_centers_ # The 4 centroïds estimated

## Prediction of the labels of new instances:

In [ ]:
X_new = np.array([[0, 2], [3, 2], [-3, 3], [-3, 2.5]])
kmeans.predict(X_new)

## Visualize the results

In [ ]:
plt.figure(figsize=(8, 5))
plt.scatter(X[:, 0], X[:, 1], c=y_kmeans, s=50, cmap='viridis')
centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5);

<div class="alert alert-info">
<h3>
 Your turn: fit a `KMeans` algorithm on the same data with $6$ clusters and plot the result in 2D 
 </h3>
</div>

### Finding the optimal number of clusers: Inertia

In [ ]:
kmeans_per_k = [KMeans(n_clusters=k, random_state=42, n_init=10).fit(X)
                for k in range(1, 10)]
inertias = [model.inertia_ for model in kmeans_per_k]
plt.figure(figsize=(8, 3.5))
plt.plot(range(1, 10), inertias, "bo-")
plt.xlabel("$k$", fontsize=14)
plt.ylabel("Inertia", fontsize=14)
plt.annotate('Elbow',
             xy=(4, inertias[3]),
             xytext=(0.55, 0.55),
             textcoords='figure fraction',
             fontsize=16,
             arrowprops=dict(facecolor='black', shrink=0.1)
            )
plt.axis([1, 8.5, 0, 1300])
plt.show()

### Finding the optimal number of clusers: Silhouette Score

In [ ]:
from sklearn.metrics import silhouette_score
silhouette_score(X, kmeans.labels_)

In [ ]:
silhouette_scores = [silhouette_score(X, model.labels_)
                     for model in kmeans_per_k[1:]]

In [ ]:
plt.figure(figsize=(8, 3))
plt.plot(range(2, 10), silhouette_scores, "bo-")
plt.xlabel("$k$", fontsize=14)
plt.ylabel("Silhouette score", fontsize=14)
plt.axis([1.8, 8.5, 0.3, 0.7])
plt.show()

## Hierarchical Clustering
### Fitting the model

In [ ]:
from sklearn.cluster import AgglomerativeClustering
agglo = AgglomerativeClustering(n_clusters=5)
agglo.fit(X)

In [ ]:
labels = agglo.fit_predict(X)
plt.scatter(X[:, 0], X[:, 1], c=labels,
            s=50, cmap='viridis');

In [ ]:
# setting distance_threshold=0 ensures we compute the full tree.
agglo2 = AgglomerativeClustering(distance_threshold=0, n_clusters=None)
agglo2.fit(X)
labels = agglo2.fit_predict(X)
plt.scatter(X[:, 0], X[:, 1], c=labels,
            s=50, cmap='viridis');

#### Dendrogram plot (with `scipy`)

We can use Scipy's `hierarchy.linkage()` to form clusters and plot them with `hierarchy.dendrogram()`:
The three big dendrograms correspond to the clusters with largest distances among them.

In [ ]:
from scipy.cluster import hierarchy

hierarchy.dendrogram(hierarchy.linkage(X, 'single'),
            orientation='top',
            distance_sort='descending',
            show_leaf_counts=True)

# Plotting a horizontal line based on the first biggest distance between clusters 
plt.axhline(0.9, color='red', linestyle='--'); 
# Plotting a horizontal line based on the second biggest distance between clusters 
plt.axhline(.62, color='crimson'); 
plt.show()

This example shows how the Dendrogram is only a reference when used to choose the number of clusters. We already know that we have 4 clusters in the dataset, but if we were to determine their number by the Dendrogram, 3 would be our first option, and 4 would be our second option.